In [1]:
import pandas as pd
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, average_precision_score, mean_squared_error, r2_score, mean_absolute_error
from scipy.stats import pearsonr

import numpy as np

# preds

# Load the training set of meta-model
ct_chemberta2_valid2 = pd.read_csv('./chemberta2/results/clintox/chemberta2_valid2_clintox_3_predictions.csv')
ct_molformer_valid2 = pd.read_csv('./molformer/results/clintox/molformer_valid2_clintox_3_epoch26.csv')
ct_molbert_valid2 = pd.read_csv('./molbert/results/cilntox/molbert_valid2_clintox_3.csv')

# Load the test data for each model
ct_chemberta2_test = pd.read_csv('./chemberta2/results/clintox/chemberta2_test_clintox_3_predictions.csv')
ct_molformer_test = pd.read_csv('./molformer/results/clintox/molformer_test_clintox_3_epoch26.csv')
ct_molbert_test = pd.read_csv('./molbert/results/cilntox/molbert_test_clintox_3.csv')

# features

# Load the features from chemberta
ct_chemberta2_features_valid2 = pd.read_csv('./chemberta2/features/clintox/chemberta2_valid2_clintox_3_features.csv')
ct_chemberta2_features_test = pd.read_csv('./chemberta2/features/clintox/chemberta2_test_clintox_3_features.csv')

# Load the features from molformer
ct_molformer_features_valid2 = pd.read_csv('./molformer/features/clintox/molformer_valid2_clintox_3_features.csv')
ct_molformer_features_test = pd.read_csv('./molformer/features/clintox/molformer_test_clintox_3_features.csv')

# Load the features from molbert
ct_molbert_features_valid2 = pd.read_csv('./molbert/features/clintox/molbert_valid2_clintox_3_features.csv')
ct_molbert_features_test = pd.read_csv('./molbert/features/clintox/molbert_test_clintox_3_features.csv')


For Clintox (Classification)

In [2]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score

# Preparing the actual and predicted values
# Chemberta2
ct_chemberta_actual = ct_chemberta2_test['CT_TOX']
ct_chemberta_pred = ct_chemberta2_test['y_pred']
ct_chemberta_probs = ct_chemberta2_test[['softmax_class_0_prob', 'softmax_class_1_prob']]

# Molformer
ct_molformer_actual = ct_molformer_test['Actual']
ct_molformer_pred = (ct_molformer_test['Prob_Class_1'] > 0.5).astype(int)
ct_molformer_probs = ct_molformer_test[['Prob_Class_0', 'Prob_Class_1']]

# Molbert
ct_molbert_actual = ct_molbert_test['target']
ct_molbert_pred = ct_molbert_test['pred']
ct_molbert_probs = ct_molbert_test['prob']

# Calculating metrics
ct_metrics_results = {}

for model_name, actual, pred, probs in [("Chemberta2", ct_chemberta_actual, ct_chemberta_pred, ct_chemberta_probs['softmax_class_1_prob']),
                                         ("Molformer", ct_molformer_actual, ct_molformer_pred, ct_molformer_probs['Prob_Class_1']),
                                         ("Molbert", ct_molbert_actual, ct_molbert_pred, ct_molbert_probs)]:
    ct_metrics_results[model_name] = {
        "Accuracy": accuracy_score(actual, pred),
        "F1 Score": f1_score(actual, pred),
        "ROC-AUC": roc_auc_score(actual, probs),
        "PR-AUC": average_precision_score(actual, probs)
    }

ct_metrics_results

{'Chemberta2': {'Accuracy': 0.9731543624161074,
  'F1 Score': 0.8,
  'ROC-AUC': 0.9964028776978417,
  'PR-AUC': 0.9540404040404039},
 'Molformer': {'Accuracy': 0.9328859060402684,
  'F1 Score': 0.375,
  'ROC-AUC': 0.9266187050359712,
  'PR-AUC': 0.5340980864486556},
 'Molbert': {'Accuracy': 0.9172413793103448,
  'F1 Score': 0.14285714285714285,
  'ROC-AUC': 0.9133333333333333,
  'PR-AUC': 0.46665875863902184}}

In [3]:
# Determine the 'smiles' values in chemberta that are not in molbert
missing_smiles = set(ct_chemberta2_valid2['smiles']) - set(ct_molbert_valid2['smiles'])

# Find the indices of these missing 'smiles' in the chemberta DataFrame
missing_indices = ct_chemberta2_valid2.index[ct_chemberta2_valid2['smiles'].isin(missing_smiles)].tolist()

# Drop the rows with these missing indices for chemberta and molformer
ct_chemberta2_valid2 = ct_chemberta2_valid2.drop(missing_indices)
ct_molformer_valid2 = ct_molformer_valid2.drop(missing_indices)
ct_chemberta2_features_valid2 = ct_chemberta2_features_valid2.drop(missing_indices)
ct_molformer_features_valid2 = ct_molformer_features_valid2.drop(missing_indices)
ct_molbert_features_valid2 = ct_molbert_features_valid2.drop(missing_indices)

# do the same for test sets
missing_smiles = set(ct_chemberta2_test['smiles']) - set(ct_molbert_test['smiles'])
missing_indices = ct_chemberta2_test.index[ct_chemberta2_test['smiles'].isin(missing_smiles)].tolist()

ct_chemberta2_test = ct_chemberta2_test.drop(missing_indices)
ct_molformer_test = ct_molformer_test.drop(missing_indices)
ct_chemberta2_features_test = ct_chemberta2_features_test.drop(missing_indices)
ct_molformer_features_test = ct_molformer_features_test.drop(missing_indices)
ct_molbert_features_test = ct_molbert_features_test.drop(missing_indices)

In [4]:
# check shapes
print(ct_chemberta2_valid2.shape)
print(ct_molformer_valid2.shape)
print(ct_molbert_valid2.shape)
print(ct_chemberta2_features_valid2.shape)
print(ct_molformer_features_valid2.shape)
print(ct_molbert_features_valid2.shape)

(295, 8)
(295, 5)
(295, 4)
(295, 386)
(295, 769)
(295, 769)


In [5]:
ct_chemberta2_valid2.reset_index(drop=True, inplace=True)
ct_y_ensemble_valid2 = ct_chemberta2_valid2['CT_TOX']

# create a new dataframe with one column of ct_chemberta2_valid2['softmax_class_1_prob']
ct_chemberta2_prob = pd.DataFrame({'chemberta2': ct_chemberta2_valid2['softmax_class_1_prob']})
ct_chemberta2_prob.reset_index(drop=True, inplace=True)

# create a new dataframe with one column of ct_molformer_valid2['Prob_Class_1']
ct_molformer_prob = pd.DataFrame({'molformer': ct_molformer_valid2['Prob_Class_1']})
ct_molformer_prob.reset_index(drop=True, inplace=True)

# create a new dataframe with one column of ct_molbert_valid2['Probabilities']
ct_molbert_prob = pd.DataFrame({'molbert': ct_molbert_valid2['prob']})
ct_molbert_prob.reset_index(drop=True, inplace=True)

# concatenate the three dataframes
ct_prob = pd.concat([ct_chemberta2_prob, ct_molformer_prob, ct_molbert_prob], axis=1)

# do the same for features ct_chemberta2_features_valid2.iloc[:, 2:]
ct_chemberta2_features = pd.DataFrame(ct_chemberta2_features_valid2.iloc[:, 2:])
ct_chemberta2_features.reset_index(drop=True, inplace=True)
ct_molformer_features = pd.DataFrame(ct_molformer_features_valid2.iloc[:, 1:])
ct_molformer_features.reset_index(drop=True, inplace=True)
ct_molbert_features = pd.DataFrame(ct_molbert_features_valid2.iloc[:, 1:])
ct_molbert_features.reset_index(drop=True, inplace=True)

ct_features = pd.concat([ct_chemberta2_features, ct_molformer_features, ct_molbert_features], axis=1)

# combine the features and probabilities
ct_X_ensemble_valid2 = pd.concat([ct_prob, ct_features], axis=1)

In [6]:
ct_chemberta2_test.reset_index(drop=True, inplace=True)
ct_y_ensemble_test = ct_chemberta2_test['CT_TOX']

# do the same for test probs and features
ct_chemberta2_prob = pd.DataFrame({'chemberta2': ct_chemberta2_test['softmax_class_1_prob']})
ct_chemberta2_prob.reset_index(drop=True, inplace=True)
ct_molformer_prob = pd.DataFrame({'molformer': ct_molformer_test['Prob_Class_1']})
ct_molformer_prob.reset_index(drop=True, inplace=True)
ct_molbert_prob = pd.DataFrame({'molbert': ct_molbert_test['prob']})
ct_molbert_prob.reset_index(drop=True, inplace=True)

ct_prob = pd.concat([ct_chemberta2_prob, ct_molformer_prob, ct_molbert_prob], axis=1)

ct_chemberta2_features = pd.DataFrame(ct_chemberta2_features_test.iloc[:, 2:])
ct_chemberta2_features.reset_index(drop=True, inplace=True)
ct_molformer_features = pd.DataFrame(ct_molformer_features_test.iloc[:, 1:])
ct_molformer_features.reset_index(drop=True, inplace=True)
ct_molbert_features = pd.DataFrame(ct_molbert_features_test.iloc[:, 1:])
ct_molbert_features.reset_index(drop=True, inplace=True)
ct_features = pd.concat([ct_chemberta2_features, ct_molformer_features, ct_molbert_features], axis=1)

ct_X_ensemble_test = pd.concat([ct_prob, ct_features], axis=1)

In [7]:
# use standard scaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
ct_X_ensemble_valid2_scaled = scaler.fit_transform(ct_X_ensemble_valid2)
ct_X_ensemble_test_scaled = scaler.transform(ct_X_ensemble_test)

# transform back to dataframe
ct_X_ensemble_valid2_scaled = pd.DataFrame(ct_X_ensemble_valid2_scaled, columns=ct_X_ensemble_valid2.columns)
ct_X_ensemble_test_scaled = pd.DataFrame(ct_X_ensemble_test_scaled, columns=ct_X_ensemble_test.columns)


In [8]:
# save the scaled data to csv
ct_X_ensemble_valid2_scaled.to_csv('./processed_data/clintox_X_ensemble_valid2_scaled_rawpreds.csv', index=False)
ct_X_ensemble_test_scaled.to_csv('./processed_data/clintox_X_ensemble_test_scaled_rawpreds.csv', index=False)

# save the target values to csv
ct_y_ensemble_valid2.to_csv('./processed_data/clintox_y_ensemble_valid2.csv', index=False)
ct_y_ensemble_test.to_csv('./processed_data/clintox_y_ensemble_test.csv', index=False)

In [9]:
# use lasso regression to train the ensemble model
from sklearn.linear_model import LogisticRegressionCV

# cross validation for strength of regularization
lasso_cv = LogisticRegressionCV(cv=5, penalty='l1', solver='liblinear', max_iter=5000, random_state=0, scoring='roc_auc')

# Fit the model
lasso_cv.fit(ct_X_ensemble_valid2_scaled, ct_y_ensemble_valid2)

# Predict the test set
ct_lasso_pred = lasso_cv.predict(ct_X_ensemble_test_scaled)
ct_lasso_probs = lasso_cv.predict_proba(ct_X_ensemble_test_scaled)[:, 1]

# Calculate the metrics
ct_lasso_metrics = {
    "Accuracy": accuracy_score(ct_y_ensemble_test, ct_lasso_pred),
    "F1 Score": f1_score(ct_y_ensemble_test, ct_lasso_pred),
    "ROC-AUC": roc_auc_score(ct_y_ensemble_test, ct_lasso_probs),
    "PR-AUC": average_precision_score(ct_y_ensemble_test, ct_lasso_probs)
}

ct_lasso_metrics

{'Accuracy': 0.9724137931034482,
 'F1 Score': 0.8,
 'ROC-AUC': 0.9962962962962962,
 'PR-AUC': 0.954040404040404}

In [10]:
coefs = pd.Series(lasso_cv.coef_[0], index=ct_X_ensemble_valid2.columns)

# Filter to get the selected features
selected_features = coefs[coefs != 0].index.tolist()

print("Selected Features:", selected_features)

Selected Features: ['chemberta2']


In [11]:
from group_lasso import LogisticGroupLasso
import numpy as np

# Create an array that specifies the group for each feature
# Assuming the number of features in your dataset:
n_features = ct_X_ensemble_valid2_scaled.shape[1]
groups = np.zeros(n_features, dtype=int)
groups[:3] = 1  # First three features as one group
groups[3:] = 2  # Rest of the features as another group

# Initialize the Logistic Group Lasso model
group_lasso = LogisticGroupLasso(
    groups=groups,
    group_reg=0.05,  # Regularization strength for the groups
    l1_reg=0,        # No L1 regularization
    scale_reg='none', # Do not automatically scale the regularization
    supress_warning=True,
    tol=1e-2,
    random_state=0
)

# Fit the model
group_lasso.fit(ct_X_ensemble_valid2_scaled, ct_y_ensemble_valid2)

# Predict the test set
ct_group_lasso_pred = group_lasso.predict(ct_X_ensemble_test_scaled)
ct_group_lasso_probs = group_lasso.predict_proba(ct_X_ensemble_test_scaled)[:, 1]

ct_two_groups_lasso_metrics = {
    "Accuracy": accuracy_score(ct_y_ensemble_test, ct_group_lasso_pred),
    "F1 Score": f1_score(ct_y_ensemble_test, ct_group_lasso_pred),
    "ROC-AUC": roc_auc_score(ct_y_ensemble_test, ct_group_lasso_probs),
    "PR-AUC": average_precision_score(ct_y_ensemble_test, ct_group_lasso_probs)
}

ct_two_groups_lasso_metrics

{'Accuracy': 0.896551724137931,
 'F1 Score': 0.5454545454545454,
 'ROC-AUC': 0.9355555555555556,
 'PR-AUC': 0.8588626739261948}

In [12]:
from group_lasso import LogisticGroupLasso
import numpy as np

# Create an array that specifies the group for each feature
# Assuming the number of features in your dataset:
n_features = ct_X_ensemble_valid2_scaled.shape[1]
groups = np.zeros(n_features, dtype=int)

# Setting the groups according to the new specification:
groups[:3] = 1      # First three features as one group
groups[3:3+384] = 2 # Next 384 features as second group
groups[3+384:3+384+768] = 3 # Next 768 features as third group
groups[3+384+768:] = 4 # Remaining 768 features as fourth group

# Initialize the Logistic Group Lasso model
group_lasso = LogisticGroupLasso(
    groups=groups,
    group_reg=0.05,  # Regularization strength for the groups
    l1_reg=0,        # No L1 regularization
    scale_reg='none', # Do not automatically scale the regularization
    supress_warning=True,
    tol=1e-2,
    random_state=0
)

# Fit the model
group_lasso.fit(ct_X_ensemble_valid2_scaled, ct_y_ensemble_valid2)

# Predict the test set
ct_group_lasso_pred = group_lasso.predict(ct_X_ensemble_test_scaled)
ct_group_lasso_probs = group_lasso.predict_proba(ct_X_ensemble_test_scaled)[:, 1]

# Calculate the metrics
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score

ct_four_groups_lasso_metrics = {
    "Accuracy": accuracy_score(ct_y_ensemble_test, ct_group_lasso_pred),
    "F1 Score": f1_score(ct_y_ensemble_test, ct_group_lasso_pred),
    "ROC-AUC": roc_auc_score(ct_y_ensemble_test, ct_group_lasso_probs),
    "PR-AUC": average_precision_score(ct_y_ensemble_test, ct_group_lasso_probs)
}

ct_four_groups_lasso_metrics

/Users/lorrainelu/anaconda3/lib/python3.11/site-packages/group_lasso/_fista.py:114: ConvergenceWarning: The FISTA iterations did not converge to a sufficient minimum.
You used subsampling then this is expected, otherwise, try increasing the number of iterations or decreasing the tolerance.
  warnings.warn(


{'Accuracy': 0.9724137931034482,
 'F1 Score': 0.8,
 'ROC-AUC': 0.9503703703703703,
 'PR-AUC': 0.8674339300937767}

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score

# Define the model with elasticnet penalty
elastic_net_model = LogisticRegression(penalty='elasticnet', solver='saga', max_iter=5000, random_state=0)

# Use fewer discrete values for alpha and l1_ratio
alphas = [0.01, 0.1, 1, 3]  # Reduced number of points focusing on lower and mid-range
l1_ratios = [0.1, 0.5, 0.9]  # Reduced to three points, emphasizing edges and midpoint

# Convert alphas to Cs for the parameter grid (since C is the inverse of alpha)
Cs = [1/alpha for alpha in alphas]

# Create a more concise grid search using 5-fold cross-validation
params = {
    'C': Cs,
    'l1_ratio': l1_ratios
}

grid_search = GridSearchCV(elastic_net_model, param_grid=params, cv=5, scoring='roc_auc')

# Fit the grid search to the data
grid_search.fit(ct_X_ensemble_valid2_scaled, ct_y_ensemble_valid2)

# Best model after grid search
ct_best_elastic_model = grid_search.best_estimator_
print(grid_search.best_params_)

# Predict the test set
ct_elastic_pred = ct_best_elastic_model.predict(ct_X_ensemble_test_scaled)
ct_elastic_probs = ct_best_elastic_model.predict_proba(ct_X_ensemble_test_scaled)[:, 1]

# Calculate the metrics
ct_elastic_metrics = {
    "Accuracy": accuracy_score(ct_y_ensemble_test, ct_elastic_pred),
    "F1 Score": f1_score(ct_y_ensemble_test, ct_elastic_pred),
    "ROC-AUC": roc_auc_score(ct_y_ensemble_test, ct_elastic_probs),
    "PR-AUC": average_precision_score(ct_y_ensemble_test, ct_elastic_probs)
}

ct_elastic_metrics

/Users/lorrainelu/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/lorrainelu/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/lorrainelu/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/lorrainelu/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/lorrainelu/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/lorrainelu/anaconda3/lib/python3.

{'C': 0.3333333333333333, 'l1_ratio': 0.1}


{'Accuracy': 0.9517241379310345,
 'F1 Score': 0.72,
 'ROC-AUC': 0.9733333333333334,
 'PR-AUC': 0.8886759581881534}

In [14]:
# check coefficients
coefs = pd.Series(ct_best_elastic_model.coef_[0], index=ct_X_ensemble_valid2.columns)

# Filter to get the selected features
selected_features = coefs[coefs != 0].index.tolist()

print("Selected Features:", selected_features)

Selected Features: ['chemberta2', 'molformer', 'molbert', 'chemberta2_feature_4', 'chemberta2_feature_5', 'chemberta2_feature_9', 'chemberta2_feature_10', 'chemberta2_feature_13', 'chemberta2_feature_14', 'chemberta2_feature_16', 'chemberta2_feature_17', 'chemberta2_feature_19', 'chemberta2_feature_20', 'chemberta2_feature_22', 'chemberta2_feature_23', 'chemberta2_feature_25', 'chemberta2_feature_26', 'chemberta2_feature_28', 'chemberta2_feature_29', 'chemberta2_feature_31', 'chemberta2_feature_32', 'chemberta2_feature_34', 'chemberta2_feature_37', 'chemberta2_feature_39', 'chemberta2_feature_41', 'chemberta2_feature_43', 'chemberta2_feature_44', 'chemberta2_feature_46', 'chemberta2_feature_48', 'chemberta2_feature_49', 'chemberta2_feature_50', 'chemberta2_feature_52', 'chemberta2_feature_53', 'chemberta2_feature_54', 'chemberta2_feature_55', 'chemberta2_feature_57', 'chemberta2_feature_58', 'chemberta2_feature_64', 'chemberta2_feature_65', 'chemberta2_feature_66', 'chemberta2_feature_

In [15]:
ct_X_ensemble_valid2_selected = ct_X_ensemble_valid2_scaled
ct_X_ensemble_test_selected = ct_X_ensemble_test_scaled
# check shapes
print(ct_X_ensemble_valid2_selected.shape)
print(ct_X_ensemble_test_selected.shape)

(295, 1923)
(145, 1923)


In [16]:
# Initialize and train the SVM model
from sklearn.svm import SVC
import random

# Set seeds for reproducibility
np.random.seed(0)
random.seed(0)

ct_svm_model = SVC(probability=True, random_state=0)
ct_svm_model.fit(ct_X_ensemble_valid2_selected, ct_y_ensemble_valid2)

# Predict the test set
ct_svm_pred = ct_svm_model.predict(ct_X_ensemble_test_selected)
ct_svm_probs = ct_svm_model.predict_proba(ct_X_ensemble_test_selected)

# Calculate the metrics
ct_svm_metrics = {
    'Accuracy': accuracy_score(ct_y_ensemble_test, ct_svm_pred),
    'F1 Score': f1_score(ct_y_ensemble_test, ct_svm_pred),
    'ROC-AUC': roc_auc_score(ct_y_ensemble_test, ct_svm_probs[:, 1]),
    "PR-AUC": average_precision_score(ct_y_ensemble_test, ct_svm_probs[:, 1])
}

ct_svm_metrics

{'Accuracy': 0.9586206896551724,
 'F1 Score': 0.5714285714285714,
 'ROC-AUC': 0.9518518518518518,
 'PR-AUC': 0.8479001825928181}

In [17]:
# initailize and use a 5-fold cross-validation to tune the hyperparameters of a random forest model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

ct_rf_model = RandomForestClassifier(random_state=0)

ct_rf_model.fit(ct_X_ensemble_valid2_selected, ct_y_ensemble_valid2)

# Predict the test set
ct_rf_best_pred = ct_rf_model.predict(ct_X_ensemble_test_selected)
ct_rf_best_probs = ct_rf_model.predict_proba(ct_X_ensemble_test_selected)

# Calculate the metrics
ct_rf_best_metrics = {
    "Accuracy": accuracy_score(ct_y_ensemble_test, ct_rf_best_pred),
    "F1 Score": f1_score(ct_y_ensemble_test, ct_rf_best_pred),
    "ROC-AUC": roc_auc_score(ct_y_ensemble_test, ct_rf_best_probs[:, 1]),
    "PR-AUC": average_precision_score(ct_y_ensemble_test, ct_rf_best_probs[:, 1])
}

ct_rf_best_metrics

{'Accuracy': 0.9586206896551724,
 'F1 Score': 0.5714285714285714,
 'ROC-AUC': 0.9537037037037037,
 'PR-AUC': 0.7985493230174081}

In [18]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, make_scorer
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.early_stop import no_progress_loss

# Set seeds for reproducibility
np.random.seed(0)
random.seed(0)

# Define the hyperparameter space using continuous distributions
ct_xgb_hyperopt_space = {
    'n_estimators': hp.quniform('n_estimators', 50, 200, 50),
    'max_depth': hp.quniform('max_depth', 3, 7, 2),
    'learning_rate': hp.uniform('learning_rate', 0.001, 0.3),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0)
}

# Convert float outputs of hp.quniform to int for certain parameters
def objective(params):
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])
    
    model = xgb.XGBClassifier(**params, random_state=0)
    
    # Cross-validated AUC score as the objective
    roc_auc = make_scorer(roc_auc_score, response_method=None)
    score = cross_val_score(model, ct_X_ensemble_valid2_selected, ct_y_ensemble_valid2, scoring=roc_auc, cv=5)
    
    # Minimize the negative ROC AUC score
    return {'loss': -score.mean(), 'status': STATUS_OK}

# Run the Bayesian optimization
trials = Trials()
ct_xgb_best_params = fmin(fn=objective, 
                          space=ct_xgb_hyperopt_space, 
                          algo=tpe.suggest, 
                          max_evals=50, 
                          trials=trials,
                          rstate=np.random.default_rng(0),  # Seed for hyperopt
                          early_stop_fn=no_progress_loss(10))

print("Best hyperparameters:", ct_xgb_best_params)

 20%|██        | 10/50 [00:28<01:53,  2.83s/trial, best loss: -0.9099999999999999]
Best hyperparameters: {'colsample_bytree': 0.9526919134012697, 'learning_rate': 0.25068293229696836, 'max_depth': 4.0, 'n_estimators': 100.0, 'subsample': 0.8411318103731124}


In [19]:
# Convert parameters obtained from Hyperopt to the correct data type
ct_xgb_best_params['n_estimators'] = int(ct_xgb_best_params['n_estimators'])
ct_xgb_best_params['max_depth'] = int(ct_xgb_best_params['max_depth'])

# Initialize and train the XGBoost model with the best parameters
ct_xgb_model = xgb.XGBClassifier(**ct_xgb_best_params, random_state=0)
ct_xgb_model.fit(ct_X_ensemble_valid2_selected, ct_y_ensemble_valid2)

# Predict the test set
ct_xgb_best_pred = ct_xgb_model.predict(ct_X_ensemble_test_selected)
ct_xgb_best_probs = ct_xgb_model.predict_proba(ct_X_ensemble_test_selected)

# Calculate the metrics
ct_xgb_best_metrics = {
    "Accuracy": accuracy_score(ct_y_ensemble_test, ct_xgb_best_pred),
    "F1 Score": f1_score(ct_y_ensemble_test, ct_xgb_best_pred),
    "ROC-AUC": roc_auc_score(ct_y_ensemble_test, ct_xgb_best_probs[:, 1]),
    "PR-AUC": average_precision_score(ct_y_ensemble_test, ct_xgb_best_probs[:, 1])
}

ct_xgb_best_metrics

{'Accuracy': 0.9862068965517241,
 'F1 Score': 0.8888888888888888,
 'ROC-AUC': 0.9903703703703703,
 'PR-AUC': 0.9354545454545454}

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
import numpy as np
from sklearn.metrics import roc_auc_score
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.early_stop import no_progress_loss

# Set seeds for reproducibility
np.random.seed(0)
random.seed(0)
torch.manual_seed(0)

# Define the neural network model
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_layers, num_neurons, dropout_rate):
        super(SimpleNN, self).__init__()
        layers = [nn.Linear(input_size, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        for _ in range(num_layers - 1):
            layers += [nn.Linear(num_neurons, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        layers += [nn.Linear(num_neurons, 1), nn.Sigmoid()]
        
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

# Objective function for Bayesian optimization
def objective(params):
    kf = KFold(n_splits=5)
    roc_aucs = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        train_dataset = TensorDataset(torch.tensor(X_train.values.astype(np.float32)), 
                                      torch.tensor(y_train.values.astype(np.float32)).unsqueeze(1))
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

        model = SimpleNN(input_size=X_train.shape[1], num_layers=int(params['num_layers']), 
                         num_neurons=int(params['num_neurons']), dropout_rate=params['dropout_rate'])
        criterion = nn.BCELoss()
        optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

        model.train()
        for epoch in range(100):
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val.values.astype(np.float32))
            y_val_tensor = torch.tensor(y_val.values.astype(np.float32)).unsqueeze(-1)
            outputs = model(X_val_tensor)
            roc_auc = roc_auc_score(y_val_tensor.numpy(), outputs.numpy())
            roc_aucs.append(roc_auc)

    avg_roc_auc = np.mean(roc_aucs)
    return {'loss': -avg_roc_auc, 'status': STATUS_OK}  # Maximize ROC AUC by minimizing the negative ROC AUC

# Hyperparameter space
space = {
    'num_layers': hp.quniform('num_layers', 1, 5, 1),
    'num_neurons': hp.quniform('num_neurons', 16, 256, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.01)),
    'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.5)
}

X = ct_X_ensemble_valid2_selected
y = ct_y_ensemble_valid2

# Run Bayesian optimization
trials = Trials()
ct_nn_best_params = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trials,
            rstate=np.random.default_rng(0),  # Seed for hyperopt
            early_stop_fn=no_progress_loss(10))

print("Best hyperparameters:", ct_nn_best_params)


 66%|██████▌   | 33/50 [03:48<01:57,  6.92s/trial, best loss: -0.9694053744997142]
Best hyperparameters: {'dropout_rate': 0.05859227318975018, 'learning_rate': 0.0014876330911043408, 'num_layers': 4.0, 'num_neurons': 227.0}


In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.metrics import roc_auc_score

torch.manual_seed(0)

# Define the neural network model again
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_layers, num_neurons, dropout_rate):
        super(SimpleNN, self).__init__()
        layers = [nn.Linear(input_size, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        for _ in range(num_layers - 1):
            layers += [nn.Linear(num_neurons, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        layers += [nn.Linear(num_neurons, 1), nn.Sigmoid()]
        
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

# Convert parameters to the correct format if necessary
ct_nn_best_params = {
    'num_layers': int(ct_nn_best_params['num_layers']),  # Extracted from Bayesian optimization results
    'num_neurons': int(ct_nn_best_params['num_neurons']),  # Extracted from Bayesian optimization results
    'dropout_rate': ct_nn_best_params['dropout_rate'],  # Extracted from Bayesian optimization results
    'learning_rate': ct_nn_best_params['learning_rate']  # Extracted from Bayesian optimization results
}

# Prepare datasets
X_train_tensor = torch.tensor(ct_X_ensemble_valid2_selected.values.astype(np.float32))
y_train_tensor = torch.tensor(ct_y_ensemble_valid2.values.astype(np.float32)).unsqueeze(1)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

X_test_tensor = torch.tensor(ct_X_ensemble_test_selected.values.astype(np.float32))
y_test_tensor = torch.tensor(ct_y_ensemble_test.values.astype(np.float32)).unsqueeze(1)

# Initialize the model
model = SimpleNN(input_size=ct_X_ensemble_valid2_selected.shape[1], num_layers=ct_nn_best_params['num_layers'], 
                 num_neurons=ct_nn_best_params['num_neurons'], dropout_rate=ct_nn_best_params['dropout_rate'])

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=ct_nn_best_params['learning_rate'])

# Training loop
model.train()
for epoch in range(100):  # Number of epochs can be adjusted
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

# Evaluation on test set
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    predictions = (outputs > 0.5).float()

    # Calculate metrics
    accuracy = accuracy_score(y_test_tensor.numpy(), predictions.numpy())
    f1 = f1_score(y_test_tensor.numpy(), predictions.numpy())
    roc_auc = roc_auc_score(y_test_tensor.numpy(), outputs.numpy())
    pr_auc = average_precision_score(y_test_tensor.numpy(), outputs.numpy())

    ct_nn_metrics = {
        'Accuracy': accuracy,
        'F1 Score': f1,
        'ROC-AUC': roc_auc,
        'PR-AUC': pr_auc
    }

ct_nn_metrics

{'Accuracy': 0.9586206896551724,
 'F1 Score': 0.7272727272727273,
 'ROC-AUC': 0.9477777777777778,
 'PR-AUC': 0.7598710317460318}

In [22]:
# report all the metrics for ct
ct_metrics_results["LASSO"] = ct_lasso_metrics
ct_metrics_results["Group Lasso (Two Groups)"] = ct_two_groups_lasso_metrics
ct_metrics_results["Group Lasso (Four Groups)"] = ct_four_groups_lasso_metrics
ct_metrics_results["Elastic Net"] = ct_elastic_metrics
ct_metrics_results["SVM"] = ct_svm_metrics
ct_metrics_results["Random Forest"] = ct_rf_best_metrics
ct_metrics_results["XGBoost"] = ct_xgb_best_metrics
ct_metrics_results["Neural Network"] = ct_nn_metrics

ct_metrics_df = pd.DataFrame(ct_metrics_results).T

# keep 3 digits after the decimal point
ct_metrics_df = ct_metrics_df.round(3)

# export as csv
ct_metrics_df.to_csv('./split3_ct_metrics_rawpreds.csv')